In [11]:
# Import all the other files and necessary modules
import numpy as np
import activation
import convolution
import maxpool
import fullyconnected
import utility
import createdataset

In [12]:
# Initialise filters and weights
F1,b1,F2,b2 = convolution.init_filters()
weights = fullyconnected.init_weights()

In [13]:
# Generate dataset X and Y
X = createdataset.create_images() / 255             # Data is normalised by dividing each value by 255.. so finally each value is between 0 and 1
Y = np.zeros((1200,3), dtype = int)
for i in range(400):
    Y[i,0] = 1
for i in range(400,800):
    Y[i,1] = 1
for i in range(800,1200):
    Y[i,2] = 1

In [14]:
def create_minibatches(X, Y, batchsize):
    assert X.shape[0] == Y.shape[0]
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    for start_idx in range(0, X.shape[0] - batchsize + 1, batchsize):
        excerpt = indices[start_idx:start_idx + batchsize]
        yield X[excerpt], Y[excerpt]

In [15]:
for i in range(70):
    
    for batch in create_minibatches(X, Y, batchsize = 24):
        X_mini, Y_mini = batch
    
    Z1, cache_conv1 = convolution.forward(X_mini,F1,b1,stride=1,pad=1)                    # First layer of convolution
    A1 = activation.relu_frwrd(Z1)                                                        # Relu activation is used
    A1m, cache_maxpool1 = maxpool.forward(A1, f=2, stride=2)                              # First layer of maxpool
    Z2, cache_conv2 = convolution.forward(A1m,F2,b2, stride=1, pad=1)                     # Second layer of convolution
    A2 = activation.relu_frwrd(Z2)                                                        # Relu activation is used
    A2m, cache_maxpool2 = maxpool.forward(A2, f=2, stride=2)                              # Second layer of maxpool
    X_f = utility.flatten_forward(A2m)                                                    # Flatten the layer in order to feed to fully connected layer
    cache_fc = fullyconnected.forward(X_f, weights)                                       # Fully connected forward propagation
    cost = fullyconnected.compute_cost(cache_fc['A4'],Y_mini.T)                           # Compute cost
    error = np.mean(np.square(cache_fc['A4']-Y_mini.T))                                   # Compute error
    weights, dA = fullyconnected.backward(cache_fc,Y_mini.T,weights,alpha = 0.05)         # Fully connected backward propagation
    dA = utility.flatten_backward(dA,A2m.shape)                                           # De-flatten the layer to perform CNN and Maxpool backprop
    dA = maxpool.backward(dA,cache_maxpool2)                                              # Maxpool 2nd layer backprop
    dA, dF2, db2 = convolution.backward(dA, cache_conv2)                                  # Conv 2nd layer backprop
    F2,b2 = convolution.updatefilters(F2,b2,dF2,db2,alpha = 0.05)                         # Update F2 and b2
    dA = maxpool.backward(dA, cache_maxpool1)                                             # Maxpool 1st layer backprop
    dA, dF1, db1 = convolution.backward(dA, cache_conv1)                                  # Conv 1st layer backprop
    F1,b1 = convolution.updatefilters(F1,b1,dF1,db1,alpha=0.05)                           # Update F1 and b1

    if i%10 == 0 :
       print('Cost after '+str(i)+' iterations is '+str(cost))
       print('Accuracy = ' + str(100-error))


Cost after 0 iterations is 0.3750024859766371
Accuracy = 99.74999834268225
Cost after 10 iterations is 0.36160990620352634
Accuracy = 99.75892672919765
Cost after 20 iterations is 0.35085080210850533
Accuracy = 99.766099465261
Cost after 30 iterations is 0.3427437597889969
Accuracy = 99.77150416014067
Cost after 40 iterations is 0.337210612086806
Accuracy = 99.77519292527546
Cost after 50 iterations is 0.33414608893357034
Accuracy = 99.77723594071095
Cost after 60 iterations is 0.33336157417778955
Accuracy = 99.77775895054815


In [20]:
# Final filters and weights is loaded to Final weights.npy file

final_weights = {}
final_weights['F1'] = np.asarray(F1)
final_weights['b1'] = np.asarray(b1)
final_weights['F2'] = np.asarray(F2)
final_weights['b2'] = np.asarray(b2)

np.save('Final weights', final_weights)
np.save('Final weights', weights)